In [1]:
import numpy as np
import pandas as pd
import scanpy.api as sc
import timeit
import glob
import os

In [2]:
# Set path prefix
path_prefix = '/soe/apblair/sc_analysis/HoC_data/human/matrices_tsv/*.csv'

In [ ]:
# Create ensembl to Hugo conversion dictionary
symbol2ensemble = pd.read_csv('/projects/sysbio/users/hzgong/pablo/cluster/brain_of_cells/v4/resources/gene_id_conversion.tsv', sep='\t', index_col=1)['ensembl']
symbol2ensemble = symbol2ensemble[symbol2ensemble.notnull()]
ensembl2symbol = dict(zip(symbol2ensemble.values,symbol2ensemble.index.values))

In [3]:
# Check how many different cell ages are present
cell_batch = set()
for batches in glob.glob(path_prefix):
    cell_batch.add(batches.split('/')[-1][2:5])

In [4]:
# Create a dictionary where the key is the cell age and the value
# is the cell batch's left ventricle data set in an anndata structure
cell_dict = {k:None for k in cell_batch}
for key in cell_dict.keys():
    for batches in glob.glob(path_prefix):
        if batches.split('/')[-1][2:5] == key and batches.split('/')[-1].split('_')[-1].split('.')[0] == 'LV':
            lv_cell = sc.read(batches, cache=True, first_column_names=True).T
            lv_cell.obs = lv_cell.obs.rename(index={key: batches.split('/')[-1].split('_')[-1].split('.')[0] + "_" + key for key in lv_cell.obs.index.tolist()})
            cell_dict[key] = lv_cell

In [7]:
for sc_data in glob.glob(path_prefix):
  
    # Pass the left ventricle data set, because it was the first anndata object created for the cell batch
    if sc_data.split('/')[-1].split('.')[0].split('_')[-1] == 'LV':
        pass
    
    # Create directory path and denote by cell type and age
    directory = '/soe/apblair/sc_analysis/' + sc_data.split('/')[-1].split('.')[0]
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    
    for age in cell_dict.keys():
        if age == sc_data.split('/')[-1][2:5]:
            
            # Create anndata structure for cell batch data
            adata = sc.read(sc_data, cache=True, first_column_names=True).T
            
            # Add batch name to cell barcode
            adata.obs = adata.obs.rename(index={key: sc_data.split('/')[-1].split('.')[0].split('_')[-1] + "_" + key for key in adata.obs.index.tolist()})

            # Add metadata batch information to anndata object
            adata.obs['batch_name'] = sc_data.split('/')[-1].split('.')[0]
    break

11W
